In [1]:
import pandas as pd
import re

In [2]:
Data = pd.read_csv("JobDataTranslated.csv")
keyWords = pd.read_csv("words_set_long.csv")
keyWords_short = pd.read_csv("words_set_short.csv")
Data=Data[:-20]

In [3]:
# just a thing to clean input
#Data['Job Description'].apply(lambda x: x.lower())

def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
Data['Cleaned Job Description'] = Data['Job Description'].apply(clean)

from string import punctuation

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

#Data['Cleaned Job Description'] = Data['Cleaned Job Description'].map(remove_punctuation)

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].apply(lambda x: x.lower())

In [4]:
kWords=[keyWords_short['0'][i] for i in range(len(keyWords_short))]
#kWords=[keyWords['0'][i] for i in range(len(keyWords))]

In [5]:
#Data['Cleaned Job Description']

In [6]:

""" this thing allows us to find all words start indexes"""
def find_all(text, sub):
    start = 0
    while True:
        start = text.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

"""all words start indexes for list of words"""
def find_all_in_list(text, subs: list):
    l=[]
    for i in subs:
        l.append(list(find_all(text,i)))
    m = [l[j][k] for j in range(len(l)) for k in range(len(l[j]))]
    return m


"gives a sentence for an index of word inside"
def get_sentence(num, text ):
    l= find_all_in_list(text, ['.','!','?',';'])
    if (len(l)==0):
        return -1
    
    l.sort()
    if (num <l[0]):
        return text[0:l[0]]
    elif (num>l[-1]):
        return text[l[-1]+1:]
    
    x=[i for i in range(len(l)) if l[i]<num and l[i+1]>num]
    return (text[l[x[0]]+1:l[x[0]+1]])


In [7]:
l = []
for d in Data['Cleaned Job Description'] :
    for i in kWords:
        if d.find(i)!=-1:
            #print(d.find(i),d.find(i)+len(i), get_sentence(d.find(i),d) ,'hui',i)
            s = (get_sentence(d.find(i),d))
            skls = [(s.find(i),s.find(i)+len(i), "SKILL")]
            if (s!=-1):
                l.append((s ,{"entities": skls}))

In [8]:
l[0:10]

[(' if you are also curious about market developments, would you like to hear more about projects within the industry and are you able to translate this information into opportunities for brunel, then a role as a sales consultant is perfect for you',
  {'entities': [(210, 226, 'SKILL')]}),
 (' together with your team you operate the fields: maintenance & asset management, industrial automation, supply chain & logistics and innovation & development',
  {'entities': [(63, 79, 'SKILL')]}),
 (' so you have at least: a college degree from a commercial or technical related study at least 1 year of experience in sales and preferably in job placement a lot of ambition to grow in your profession a representative attitude strongly developed communication skills and a lot of persuasiveness and a valid driver license b',
  {'entities': [(236, 259, 'SKILL')]}),
 (' so you have at least: a college degree from a commercial or technical related study at least 1 year of experience in sales and preferab

In [9]:
len(l)

7183

In [15]:
import pickle

with open("training_set_1.txt", "wb") as fp:   #Pickling
    pickle.dump(l, fp)
    
#with open("test.txt", "rb") as fp:   # Unpickling
#b = pickle.load(fp)

In [13]:
with open("trial1.txt", "rb") as fp:
    b = pickle.load(fp)
    
b[0:10]

[(' if you are also curious about market developments, would you like to hear more about projects within the industry and are you able to translate this information into opportunities for brunel, then a role as a sales consultant is perfect for you',
  {'entities': [(210, 226, 'SKILL')]}),
 (' together with your team you operate the fields: maintenance & asset management, industrial automation, supply chain & logistics and innovation & development',
  {'entities': [(63, 79, 'SKILL')]}),
 (' so you have at least: a college degree from a commercial or technical related study at least 1 year of experience in sales and preferably in job placement a lot of ambition to grow in your profession a representative attitude strongly developed communication skills and a lot of persuasiveness and a valid driver license b',
  {'entities': [(236, 259, 'SKILL')]}),
 (' so you have at least: a college degree from a commercial or technical related study at least 1 year of experience in sales and preferab